In [3]:
import os
from openai import *
import pandas as pd
import numpy as np
import pdb
import time
sys.path.append('../utilities')
from basic_utilities import *

In [4]:
with open("../../../../api_keys/openai_api.txt", "r") as password_file:  # Open in binary read mode
    
    password = password_file.read()


In [5]:
client = OpenAI(api_key=password)


In [6]:
train_test_data_path = '../../../data/processed_data/train_test_sample.csv'

train_test_data_txt = '../../../data/processed_data/train_test_sample.txt'

In [7]:
csv_to_txt(train_test_data_path, train_test_data_txt)

In [8]:
split_file(train_test_data_txt,'../../../data/processed_data' ,5000)

Chunk 1 written to ../../../data/processed_data/chunk_1.txt
Chunk 2 written to ../../../data/processed_data/chunk_2.txt
Chunk 3 written to ../../../data/processed_data/chunk_3.txt
Chunk 4 written to ../../../data/processed_data/chunk_4.txt
Chunk 5 written to ../../../data/processed_data/chunk_5.txt
Chunk 6 written to ../../../data/processed_data/chunk_6.txt
Chunk 7 written to ../../../data/processed_data/chunk_7.txt
Chunk 8 written to ../../../data/processed_data/chunk_8.txt
Chunk 9 written to ../../../data/processed_data/chunk_9.txt
Chunk 10 written to ../../../data/processed_data/chunk_10.txt
Chunk 11 written to ../../../data/processed_data/chunk_11.txt
Chunk 12 written to ../../../data/processed_data/chunk_12.txt
Chunk 13 written to ../../../data/processed_data/chunk_13.txt
Chunk 14 written to ../../../data/processed_data/chunk_14.txt
Chunk 15 written to ../../../data/processed_data/chunk_15.txt
Chunk 16 written to ../../../data/processed_data/chunk_16.txt
Chunk 17 written to ../../

In [9]:
upload_files = [f"../../../data/processed_data/chunk_{i+1}.txt" for i in range(18)]

In [10]:
# messages = [
#     {"role": "system", "content": "You are a seasoned data scientist and engineer"}
# ]

In [11]:
# #chatbot

# while True:

#     user_input = input("Ask me anything. Or type quit to disengage.")

#     if user_input == 'quit':
#         break
#     else:
#         messages.append({"role": "user", "content": user_input})
        
#         response = client.chat.completions.create(
#         model="gpt-3.5-turbo", # Choose your desired model
#         messages=messages)

#         # pdb.set_trace()

#         response_context = response.choices[0].message.content
#         print(response_context)
#         messages.append({"role": "assistant", "content": response_context})
        

### Data analyst

In [13]:
vector_store = client.vector_stores.create(
    name="store_csv_for_data_analyst_assistant",
)

In [14]:
resource_file_id = []

for chunk in upload_files:

    file = client.files.create(
      file=open(chunk, "rb"),
      purpose='assistants'
    )
    
    vector_store_file = client.vector_stores.files.create_and_poll(
        vector_store_id=vector_store.id,
        file_id=file.id,
    )
    
    resource_file_id.append(file.id)
    
    print(f"{chunk} upload status: {vector_store_file.status}")

../../../data/processed_data/chunk_1.txt upload status: completed
../../../data/processed_data/chunk_2.txt upload status: completed
../../../data/processed_data/chunk_3.txt upload status: completed
../../../data/processed_data/chunk_4.txt upload status: completed
../../../data/processed_data/chunk_5.txt upload status: completed
../../../data/processed_data/chunk_6.txt upload status: completed
../../../data/processed_data/chunk_7.txt upload status: completed
../../../data/processed_data/chunk_8.txt upload status: completed
../../../data/processed_data/chunk_9.txt upload status: completed
../../../data/processed_data/chunk_10.txt upload status: completed
../../../data/processed_data/chunk_11.txt upload status: completed
../../../data/processed_data/chunk_12.txt upload status: completed
../../../data/processed_data/chunk_13.txt upload status: completed
../../../data/processed_data/chunk_14.txt upload status: completed
../../../data/processed_data/chunk_15.txt upload status: completed
../.

In [15]:
assistant = client.beta.assistants.create(
    name="data_analysis_assistant",
    instructions="You are an expert in data analysis. You help me summarize data",
    model="gpt-4o",
    tools=[{"type": "code_interpreter", 'type': 'file_search'}],
    tool_resources={
    "code_interpreter": {"file_ids": resource_file_id}
    }
)

In [16]:
da_ASSISTANT_ID = assistant.id

In [17]:
thread = client.beta.threads.create()

In [18]:
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [19]:
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [37]:
while True:

    user_input = input("I am your data analyst assistant. I can summarize data for you. Type quit to exit")

    if user_input == 'quit':
        break
    else:
        message = client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_input)

        run = client.beta.threads.runs.create(
            thread_id=thread.id,
            assistant_id=assistant.id)

        wait_on_run(run, thread)

        messages = client.beta.threads.messages.list(
            thread_id=thread.id, order="asc", after=message.id)
        
        pretty_print(messages)
        

I am your data analyst assistant. I can summarize data for you. Type quit to exit can you tell me how many rows and columns in the resource that I uploaded to you?


# Messages
assistant: I was unable to determine the number of rows and columns directly from the uploaded files. To help with this, I'd need to extract and analyze the data from each file individually. Let me proceed to read through them to get this information.
assistant: I will now proceed to check each file one by one to provide you with the number of rows and columns they contain.
assistant: It seems that I'm unable to automatically extract the number of rows and columns directly from the files at the moment. You can help by converting them into a more straightforward text or spreadsheet format, and I can read them directly. Alternatively, if you can share a smaller sample or tell me their file formats, I can guide you on how to find this information on your own. 

If these are spreadsheets or CSV files, here's a simple way to check:

1. **Excel or Google Sheets**: Open the file in Excel or Google Sheets. The number of columns is visible along the top (A, B, C, etc.), and you can s

I am your data analyst assistant. I can summarize data for you. Type quit to exit I uploaded 18 txt files. Concatenate and read through them


# Messages
assistant: Let's proceed by concatenating the content of the uploaded text files to analyze them for the number of rows and columns. I'll gather the contents of these files incrementally.
assistant: I seem to be experiencing issues with accessing the content of the uploaded text files directly. I recommend the following options to determine the number of rows and columns quickly:

- **Using a script**: You can write a simple script in Python or any scripting language to read and concatenate these text files. Here's an example Python snippet using the `pandas` library:

  ```python
  import pandas as pd
  import os

  # Directory where files are stored
  path = 'your_directory_path_here'

  # List all files
  all_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.txt')]

  # Read each file and concatenate
  all_data = pd.concat((pd.read_csv(f, sep='\t') for f in all_files))

  # Get shape
  print(all_data.shape)  # prints (number_of_rows, number_of_colum

I am your data analyst assistant. I can summarize data for you. Type quit to exit do you have the txt files I uploaded to you?


# Messages
assistant: Yes, you uploaded a series of text files. Since there are multiple files, to analyze them collectively, I'd need to access and read their contents individually or in batches to concatenate them. I can process them sequentially to give you the consolidated data overview. Let me attempt to fetch and read the contents of each file. 

I'll go ahead and retrieve and concatenate the contents of the text files now.
assistant: It seems I cannot directly access the content of the text files at the moment. However, you can upload them again or a representative file for me to check its content and suggest specific steps to concatenate and analyze these files using scripts or data analysis tools.

If you're able to provide more context about the structure of these files, such as whether they are table-like with consistent delimiters, I can offer more targeted advice or script templates to help you process them.



I am your data analyst assistant. I can summarize data for you. Type quit to exit quit
